In [1]:
from collections import OrderedDict
import torch
import nnsight
from nnsight import NNsight

/home/yejeon/envs/circuit/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_size = 5
hidden_dims = 10
output_size = 2

# define PyTorch model
net = torch.nn.Sequential(
    OrderedDict(
        [
            ("layer1", torch.nn.Linear(input_size, hidden_dims)),
            ("layer2", torch.nn.Linear(hidden_dims, output_size)),
        ]
    )
).requires_grad_(False)

In [16]:
# wraps around a given PyTorch model to enable investigation of its internal parameters.
# This added a couple properties to each module in the model (including the root model itself). The two most important ones are .input and .output.
tiny_model = NNsight(net)

## Inside the context, how we can save activations and execute functions on the proxies

In [11]:
"""
# with keyword to enter a context-like object. 
    - This object defines logic to be run at the start of the with block, as well as logic to be run when exiting.

# being within the context - we can read from the file
with open('myfile.txt', 'r') as file:
  text = file.read()
"""
# nnsight uses contexts to enable intuitive access into the internals of a neural network. 
# Inside the context, we will be able to customize how the neural network runs. The model is actually run upon exiting the tracing context.

# random input
input = torch.rand((1, input_size))

with tiny_model.trace(input) as tracer:

    # Proxies for the eventual inputs and outputs of a module.
    # Proxy objects will only have their value at the end of a context if we call .save() on them
    input = tiny_model.layer1.input.save()
    output = tiny_model.output.save() 

print(input)
print(output)

tensor([[0.9780, 0.3155, 0.1368, 0.3514, 0.9709]])
tensor([[-0.0975,  0.1905]])


In [12]:
# nnsight handles Pytorch functions and methods within the tracing context, by creating a Proxy request for it
with tiny_model.trace(input):

    # Note we don't need to call .save() on the output,
    # as we're only using its value within the tracing context.
    l1_output = tiny_model.layer1.output

    # We do need to save the argmax tensor however,
    # as we're using it outside the tracing context.
    l1_amax = torch.argmax(l1_output, dim=1).save()

print(l1_amax[0])

tensor(3)


In [13]:
# Everything within the tracing context operates on the intervention graph. 
# Therefore, for nnsight to trace a function it must also be a part of the intervention graph.
# How do we add them to the intervention graph? Enter nnsight.apply()

## Tracer vs Invoker

In [ ]:
# When we call .trace(...), it’s actually creating two different contexts behind the scenes. 
# The first one is the tracing context that we’ve discussed previously,
# and the second one is the invoker context
# The invoker context defines the values of the .input and .output

# If we call .trace() without an input, then we can call tracer.invoke(input1) to manually create the invoker context with an input
# interventions within its context will only refer to the input in that particular invoke statement.